## Classifying text with DistilBERT and Tensorflow

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

%cd /content/gdrive/Shareddrives/Data Challenge IA PAU - Dec 2022/model

Mounted at /content/gdrive/


In [ ]:
%%capture
!python3 -m venv venv
!source venv/bin/activate

In [ ]:
!pip install tensorflow transformers pandas pickle

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.25.1-py3-none-any.whl (5.8 MB)
     |████████████████████████████████| 7.6 MB 7.1 MB/s 
     |████████████████████████████████| 182 kB 56.5 MB/s 


In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import activations, optimizers, losses
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
import pickle
import timeit

In [ ]:
!source /content/gdrive/Shareddrives/Data Challenge IA PAU - Dec 2022/model/venv/bin/activate
!pip freeze > requirements.txt

This notebook explores binary text classification using `TFDistilBertForSequenceClassification` from the 🤗Transformers library.

This notebook is broken up into 5 sections:
1. Preprocessing the data
2. Fine-tuning the model
3. Testing the model
4. Using the fine-tuned model to predict new samples
5. Saving and loading the model for future use



Let's start off by taking a look at our dataset. In this example we consider a small corpus of 10 Yelp reviews: 5 positive (class 1) and 5 negative (class 0). BERT (and its variants like DistilBERT) can be a [great tool to use when you have a shortage of training data](https://ai.googleblog.com/2018/11/open-sourcing-bert-state-of-art-pre.html) ... that said, don't expect great results with just 10 reviews! Interchanging `x` and `y` with your own dataset is recommended 🙂

In [ ]:
def load_dataset():
    df = pd.read_csv('/content/gdrive/Shareddrives/Data Challenge IA PAU - Dec 2022/data/train_processed__lmmtiz_1__stpwrd_rm_1.csv',index_col=0)

    x_data = list(df['text'])
    y_data = list(df['label'])

    return x_data, y_data

x, y = load_dataset()

# 1. Preprocessing the data


In [ ]:
MODEL_NAME = 'distilbert-base-uncased'
MAX_LEN = 40

review = x[0]

tkzr = DistilBertTokenizer.from_pretrained(MODEL_NAME)

inputs = tkzr(review, max_length=MAX_LEN, truncation=True, padding=True)

print(f'review: \'{review}\'')
print(f'input ids: {inputs["input_ids"]}')
print(f'attention mask: {inputs["attention_mask"]}')


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

review: 'I distinct disadvantage I not see first two movie series although I see lot larry cohen film fan series seem think good film judge pretty boring never get real good look maniac cop robert zdar face I see pretty grim death scene seem stage eat film not give thrill maybe I see nc17 director cut I may impressed ending car chase zdar caitlin dulany robert davi pretty intense good part movie'
input ids: [101, 1045, 5664, 20502, 1045, 2025, 2156, 2034, 2048, 3185, 2186, 2348, 1045, 2156, 2843, 6554, 9946, 2143, 5470, 2186, 4025, 2228, 2204, 2143, 3648, 3492, 11771, 2196, 2131, 2613, 2204, 2298, 29310, 2278, 8872, 2728, 1062, 7662, 2227, 102]
attention mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


Now we must apply this transformation to each review in our corpus. To do this we define a function `construct_encodings`, which maps the tokenizer to each review and aggregates them in `encodings`:

In [ ]:
def construct_encodings(x, tkzr, max_len, trucation=True, padding=True):
    return tkzr(x, max_length=max_len, truncation=trucation, padding=padding)
    
encodings = construct_encodings(x, tkzr, max_len=MAX_LEN)

The first stage of preprocessing is done! The second stage is converting our `encodings` and `y` (which holds the classes of the reviews) into a [Tensorflow Dataset object](https://www.tensorflow.org/api_docs/python/tf/data/Dataset). Below is a function to do this:



In [ ]:
def construct_tfdataset(encodings, y=None):
    if y:
        return tf.data.Dataset.from_tensor_slices((dict(encodings),y))
    else:
        # this case is used when making predictions on unseen samples after training
        return tf.data.Dataset.from_tensor_slices(dict(encodings))
    
tfdataset = construct_tfdataset(encodings, y)

The third and final preprocessing step is to create [training](https://developers.google.com/machine-learning/glossary#training-set) and [test sets](https://developers.google.com/machine-learning/glossary#test-set): 

In [ ]:
TEST_SPLIT = 0.20
BATCH_SIZE = 4

train_size = int(len(x) * (1-TEST_SPLIT))

tfdataset = tfdataset.shuffle(len(x))
tfdataset_train = tfdataset.take(train_size)
tfdataset_test = tfdataset.skip(train_size)

tfdataset_train = tfdataset_train.batch(BATCH_SIZE)
tfdataset_test = tfdataset_test.batch(BATCH_SIZE)

Our data is finally ready. Now we can do the fun part: model fitting! 🙌 🙌 🙌

# 2. Fine-tuning the model

Fine-tuning the model is as easy as instantiating a model instance, [optimizer](https://developers.google.com/machine-learning/glossary#optimizer), and [loss](https://developers.google.com/machine-learning/glossary#loss), and then compiling/fitting:

In [ ]:
N_EPOCHS = 3

model = TFDistilBertForSequenceClassification.from_pretrained(MODEL_NAME)
optimizer = optimizers.Adam(learning_rate=3e-5)
loss = losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

start = timeit.default_timer()
model.fit(tfdataset_train, batch_size=BATCH_SIZE, epochs=N_EPOCHS)
stop = timeit.default_timer()
print('Time: ', stop - start)

Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'activation_13', 'vocab_projector', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'dropout_19', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

Epoch 1/3
6000/6000 [==============================] - 298s 48ms/step - loss: 0.4687 - accuracy: 0.7735
Epoch 2/3
6000/6000 [==============================] - 285s 47ms/step - loss: 0.3349 - accuracy: 0.8567
Epoch 3/3
6000/6000 [==============================] - 283s 47ms/step - loss: 0.2277 - accuracy: 0.9150
Time:  975.8252890829999


## 3. Testing the model

Now we can use our test set to evaluate the performance of the model. Don't expect great results from such a small training set! 😅

In [ ]:
benchmarks = model.evaluate(tfdataset_test, return_dict=True, batch_size=BATCH_SIZE)
print(benchmarks)

1500/1500 [==============================] - 33s 21ms/step - loss: 0.1292 - accuracy: 0.9592
{'loss': 0.12920904159545898, 'accuracy': 0.9591666460037231}


# 4. Using the fine-tuned model to predict new samples

Now that we've trained the model, we can use it to predict new samples. Below is a function whose output is a classifier we can use for predictions:

In [ ]:

def create_predictor(model, model_name, max_len):
  tkzr = DistilBertTokenizer.from_pretrained(model_name)
  def predict_proba(text):
      x = [text]

      encodings = construct_encodings(x, tkzr, max_len=max_len)
      tfdataset = construct_tfdataset(encodings)
      tfdataset = tfdataset.batch(1)

      preds = model.predict(tfdataset).logits
      preds = activations.softmax(tf.convert_to_tensor(preds)).numpy()
      return round(preds[0][0])
    
  return predict_proba

clf = create_predictor(model, MODEL_NAME, MAX_LEN)

#test (1:negatif, 0:positif)

print(clf('I love very much this movie !'))

print(clf('I hate this movie, it makes me cry'))

1/1 [==============================] - 0s 28ms/step
0
1/1 [==============================] - 0s 34ms/step
1


## 5. Saving and loading the model for future use

We use the built-in [save](https://huggingface.co/transformers/main_classes/model.html#transformers.PreTrainedModel.save_pretrained) and [load](https://huggingface.co/transformers/main_classes/model.html#transformers.PreTrainedModel.from_pretrained) methods from 🤗Transformers to serialize the model and restore it for future use:


In [ ]:
model.save_pretrained('./model/clf')
#model.config.to_json_file("./model/clf/config.json")
with open('./model/info.pkl', 'wb') as f:
    pickle.dump((MODEL_NAME, MAX_LEN), f)

We save `MODEL_NAME` and `MAX_LEN` for tokenizing/formatting down the line. Below we see how to reload the model for future use:

In [ ]:
new_model = TFDistilBertForSequenceClassification.from_pretrained('./model/clf')

model_name, max_len = pickle.load(open('./model/info.pkl', 'rb'))

clf = create_predictor(new_model, model_name, max_len)
print(clf('this restaurant has horrible food'))

Unsuprisingly, we get the same prediction as before!

## Congrats -- you just made a binary text classifier with 🤗Transformers! 🤜💥🤛